# Login no domínio do sistemas2.utfpr.edu.br

In [ ]:
# import json
import os
import time
import base64
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from dotenv import load_dotenv
from datetime import datetime

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()
LOGIN = os.getenv('LOGIN')
PASSWORD = os.getenv('PASSWORD')
CURSO = os.getenv('CURSO')

# Diretório onde os arquivos PDF serão salvos
download_dir = os.path.join(os.getcwd(), 'Files', 'PDF', 'Ementas', CURSO)
os.makedirs(download_dir, exist_ok=True)

# Configurações do Chrome para baixar arquivos PDF automaticamente
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "plugins.always_open_pdf_externally": True,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True
}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument("--kiosk-printing")

# Obter o ano atual e o semestre com base na data de hoje
hoje = datetime.today()
ano_atual = hoje.year
semestre_atual = 1 if hoje.month <= 6 else 2

# Defina os dados do formulário
form_data = {
    "ano_periodo": ano_atual,
    "periodo_sequencial": semestre_atual
}

# Função para imprimir para PDF
def print_to_pdf(driver, download_dir, filename):
    result = driver.execute_cdp_cmd("Page.printToPDF", {
        "printBackground": True,
        "preferCSSPageSize": True
    })
    with open(os.path.join(download_dir, filename), 'wb') as file:
        file.write(base64.b64decode(result['data']))
        
# Inicialize o driver do Selenium (certifique-se de que o driver do navegador está no PATH)
service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)
url = 'https://sistemas2.utfpr.edu.br/dpls/sistema/aluno04/mpmenu.inicio'
driver.get(url)

time.sleep(1)

# Login
driver.find_element(By.XPATH, "/html/body/app-root/app-login/div/div/p-card/div/div/div/div/form/div[2]/div/input").send_keys(LOGIN)
driver.find_element(By.XPATH, "/html/body/app-root/app-login/div/div/p-card/div/div/div/div/form/div[3]/div/input").send_keys(PASSWORD)
driver.find_element(By.XPATH, '//*[@id="btn-login"]').click()

time.sleep(1)

# Selecao de curso
botao_curso = driver.find_element(By.XPATH, '//*[@id="p_CursLinha"]')
botao_curso.click()

botao_ciencia_computacao = driver.find_element(By.XPATH, '//*[@id="p_CursLinha"]/option[2]')
botao_ciencia_computacao.click()

# Extraia o nome do curso
curso_texto = botao_ciencia_computacao.text
curso_nome = curso_texto.split('[')[0].strip()


# Download dos horários no mapa dos professores.

In [ ]:
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

# exiba a data e hor ano console
print(datetime.now())

# selecao do mapa de professores
botao_mapa_professores = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="30"]'))
)
botao_mapa_professores.click()

# Aguarde até que o iframe esteja presente e seja clicável
iframe = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="if_navega"]'))
)
# Mude para o iframe
driver.switch_to.frame(iframe)

time.sleep(1)


# Preencha o campo Ano/Período
ano_periodo_element = driver.find_element(By.XPATH, '//*[@id="pi_periodoanualanonr"]')
ano_periodo_element.clear()
ano_periodo_element.send_keys(form_data["ano_periodo"])

# Preencha o campo Período Sequencial
periodo_sequencial_element = driver.find_element(By.XPATH, '//*[@id="pr_periodoanualseqnr"]')
periodo_sequencial_element.clear()
periodo_sequencial_element.send_keys(form_data["periodo_sequencial"])

# Selecione todos os departamentos do dropdown e itere sobre eles, ignorando o primeiro campo vazio
departamentos_element = Select(driver.find_element(By.XPATH, '//*[@id="pm_deptoacadnr"]'))
departamentos = [option.text for option in departamentos_element.options if option.text != ""][1:]

for departamento in departamentos:
    # Selecione o departamento
    departamentos_element.select_by_visible_text(departamento)

    time.sleep(2)

    # Aguarde até que o dropdown de professores esteja presente
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="pm_profmnemcodnr"]'))
    )

    # Selecione todos os professores do dropdown e itere sobre eles, ignorando o primeiro campo vazio
    professores_element = Select(driver.find_element(By.XPATH, '//*[@id="pm_profmnemcodnr"]'))
    professores = [option.text for option in professores_element.options if option.text != ""][1:]

    for professor in professores:
        # Selecione o professor
        professores_element.select_by_visible_text(professor)

        time.sleep(1)

        # Clique no botão Pesquisar
        botao_pesquisar = driver.find_element(By.XPATH, '//*[@id="bt_pesquisar"]')
        botao_pesquisar.click()

        # Aguarde até que a tabela de resultados esteja presente
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="div_tab1"]'))
        )

        time.sleep(1)
        
        # Extraia as informações do professor
        professor_info = driver.find_element(By.XPATH, '//*[@id="div_tab1"]/p').text.split('\n')
        nome_professor = professor_info[2].split(' - ')[1]
        ano_periodo = professor_info[0].split(' - ')[1]

        # Localize a tabela pelo seletor (neste caso, pela classe "tabela")
        tabela = driver.find_element(By.XPATH, '//*[@id="div_tab1"]/table[1]/tbody/tr[1]/td[1]/table')

        # Extraia os cabeçalhos da tabela
        cabecalhos = []
        for th in tabela.find_elements(By.TAG_NAME, 'th'):
            cabecalhos.append(th.text.strip())

        # Extraia as linhas da tabela
        disciplinas = {}
        for linha in tabela.find_elements(By.TAG_NAME, 'tr')[1:]:  # Ignora a primeira linha (cabeçalho)
            celulas = linha.find_elements(By.TAG_NAME, 'td')
            if celulas:  # Verifica se a linha contém células
                nome = celulas[2].text.strip()
                codigo = celulas[1].text.strip()
                turma = celulas[3].text.strip().split(' - ')[0]
                enquadramento = celulas[3].text.strip().split(' - ')[-1]
                disciplinas[codigo] = [
                    {
                        "nome": nome,
                        "turma": turma,
                        "enquadramento": enquadramento
                    }
                ]

        # Mapeamento de dias e horários
        dias = ["segunda-feira", "terca-feira", "quarta-feira", "quinta-feira", "sexta-feira", "sabado"]
        periodos = ["manha", "tarde", "noite"]
        horarios_map = {
            "manha": {
                "m1": {"inicio": "07:30", "fim": "08:20"},
                "m2": {"inicio": "08:20", "fim": "09:10"},
                "m3": {"inicio": "09:10", "fim": "10:00"},
                "m4": {"inicio": "10:20", "fim": "11:10"},
                "m5": {"inicio": "11:10", "fim": "12:00"},
                "m6": {"inicio": "12:00", "fim": "13:00"}
            },
            "tarde": {
                "t1": {"inicio": "13:00", "fim": "13:50"},
                "t2": {"inicio": "13:50", "fim": "14:40"},
                "t3": {"inicio": "14:40", "fim": "15:30"},
                "t4": {"inicio": "15:50", "fim": "16:40"},
                "t5": {"inicio": "16:40", "fim": "17:30"},
                "t6": {"inicio": "17:50", "fim": "18:40"}
            },
            "noite": {
                "n1": {"inicio": "18:40", "fim": "19:30"},
                "n2": {"inicio": "19:30", "fim": "20:20"},
                "n3": {"inicio": "20:20", "fim": "21:10"},
                "n4": {"inicio": "21:20", "fim": "22:10"},
                "n5": {"inicio": "22:10", "fim": "23:00"}
            }
        }

        # Crie a estrutura inicial do JSON
        horarios = {dia: {periodo: {horario: {
            "inicio": horarios_map[periodo][horario]["inicio"],
            "fim": horarios_map[periodo][horario]["fim"],
            "disciplina": "",
            "codigo": "",  # Novo campo
            "sala": "",
            "turma": ""
        } for horario in horarios_map[periodo]} for periodo in periodos} for dia in dias}
        
        # Preencha o campo "disciplina" com as informações da tabela de horários
        for dia_index, dia in enumerate(dias):
            for periodo in periodos:
                for horario in horarios_map[periodo]:
                    try:
                        disciplina = driver.find_element(By.XPATH, f'//*[@id="dv_{dia_index+2}{horario}"]').text.strip()
                        if disciplina:
                            horarios[dia][periodo][horario]["disciplina"] = disciplina
                    except:
                        pass

        # Modifique os campos do JSON conforme solicitado
        for dia in horarios:
            for periodo in horarios[dia]:
                for horario in horarios[dia][periodo]:
                    disciplina_info = horarios[dia][periodo][horario]["disciplina"]
                    if disciplina_info:
                        partes = disciplina_info.split('\n')
                        if len(partes) == 3:
                            horarios[dia][periodo][horario]["disciplina"] = partes[0]  # Nome da disciplina
                            disciplina_partes = partes[1].split(' ')
                            if len(disciplina_partes) > 1:
                                # Extrair o código da disciplina (ex: COM1009)
                                codigo = disciplina_partes[0].split('-')[0]
                                horarios[dia][periodo][horario]["codigo"] = codigo
                                horarios[dia][periodo][horario]["sala"] = disciplina_partes[-1]
                            horarios[dia][periodo][horario]["turma"] = partes[2]

        # Crie o objeto JSON
        professor_data = {
            "departamento": departamento,
            "nome_professor": nome_professor,
            "ano_periodo": ano_periodo,
            "disciplinas": disciplinas,
            "horarios": horarios
        }

        # Converta para JSON
        professor_json = json.dumps(professor_data, indent=4, ensure_ascii=False)
        # print(professor_json)

        # Crie a pasta com o nome do ano e período se não existir
        ano_periodo_dir = os.path.join('Files', 'JSON', ano_periodo.replace('/', '-'))
        os.makedirs(ano_periodo_dir, exist_ok=True)

        # Crie a pasta com o nome do departamento dentro da pasta do ano e período se não existir
        departamento_dir = os.path.join(ano_periodo_dir, departamento)
        os.makedirs(departamento_dir, exist_ok=True)

        # Salve o JSON em um arquivo dentro da pasta do departamento
        file_name = f"Horarios-{nome_professor.replace(' ', '_')}-{ano_periodo.replace('/', '-')}.json"
        file_path = os.path.join(departamento_dir, file_name)

        with open(file_path, 'w', encoding='utf-8') as json_file:
            json_file.write(professor_json)

        print(f"Arquivo JSON salvo em: {file_path}")

# exiba a data e hora no console
print(datetime.now())

# Download das ementas do curso

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Diretório onde os arquivos PDF serão salvos
download_dir = os.path.join(os.getcwd(), 'Files', 'PDF', 'Ementas', curso_nome)
os.makedirs(download_dir, exist_ok=True)

# selecao de matrizes curriculares
botao_matrizes = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="div_CarregaAjaxMenu"]/center/table/tbody/tr/td[10]'))
)
botao_matrizes.click()

time.sleep(2)

# exiba a data e hor ano console
print(datetime.now())

# Aguarde até que o iframe esteja presente e seja clicável
iframe = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="if_navega"]'))
)
# Mude para o iframe
driver.switch_to.frame(iframe)

# Iterar sobre os períodos
periodos = driver.find_elements(By.XPATH, '//*[@id="grade"]/tbody')
for p, periodo in enumerate(periodos, start=1):
    # Iterar sobre as disciplinas dentro do período
    disciplinas = periodo.find_elements(By.XPATH, f'//*[@id="grade"]/tbody[{p}]/tr')
    for i, disciplina in enumerate(disciplinas, start=1):
        try:
            # Verifique se o botão de ementa está presente
            botao_ementa = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="grade"]/tbody[{p}]/tr[{i}]/td[3]/font/a[2]'))
            )
        except:
            continue

        # Extraia o código da disciplina
        codigo_disciplina = botao_ementa.text
        botao_ementa.click()

        time.sleep(1)

        # Mude o foco para a nova janela
        original_window = driver.current_window_handle
        for window_handle in driver.window_handles:
            if window_handle != original_window:
                driver.switch_to.window(window_handle)
                break

        try:
            # Aguarde até que o botão imprimir esteja presente e seja clicável
            botao_imprimir = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="p_imprimir"]'))
            )
            # Defina o nome do arquivo PDF
            filename = f"{codigo_disciplina}.pdf"
            # Imprima para PDF
            print_to_pdf(driver, download_dir, filename)

            time.sleep(1)
        except:
            # Se houver um erro ao imprimir, feche a janela e continue para a próxima iteração
            driver.close()
            driver.switch_to.window(original_window)
            driver.switch_to.frame(iframe)
            continue

        # Feche a nova janela e volte para a janela original
        driver.close()
        driver.switch_to.window(original_window)

        # Volte para o iframe
        driver.switch_to.frame(iframe)

# exiba a data e hor ano console
print(datetime.now())